First cut at Full Vehicle Dynamics Model for Performance Trends using equations and values from Form1.vb and FrontSuspension.m. 

Here are the initial assumptions:

Use static values for LF, RF trackwidth
LR & RR trackwidth is the same as LF & RF
Use static panhard angle
Use previous regression equation for LFIC, LFAD, LFFVSA, RFIC, RFAD, RFFVSA, LRAS, RRAS based on Pitch and Roll


Vehicle Suspension Parameters

In [1]:
CGy = 5.1 #Center of Gravity y, in
CGz = 16 #Center of Gravity z, in
LFTW = 32.29 #LF Trackwidth, in
RFTW = 32.25 #RF Trackwidth, in
LRTW = LFTW #LR Trackwidth, in 
RRTW = RFTW #RR Trackwidth, in
WB = 108 #Wheelbase, in
P2CL = 20 #Panhard to Centerline, in
LA2CL = 24 #Lower Arm to Centerline, in
P2LR = LRTW - P2CL #Panhard to LR, in
LRP = 18 #LR Panhard, in
RWR = 13.5 #Rear Wheel Radius, in
PAng = -3.3/57.3 #Panhard angle, converted from deg to rad


Vehicle Weight Parameters

In [2]:
Nose = 50 #Nose Weight Pct, %
Cross = 56 #Cross Weight Pct, %
WLeft = 56 #Left Side Weight Pct, %
WRight = (100-WLeft) #Right Side Weight Pct, %
FBrakeBias = 75 #Front Brake Bias Pct, %
VehWt = 2800 #Vehicle Weight, lbs
VehM = VehWt/32.2/12 #Convert to Mass
USLFWt = 120 #Left Front Unsprung Weight, lbs
USLFM = USLFWt/32.2/12 #Left Front Unsprung Mass 
USRFWt = 120 #Right Front Unsprung Weight, lbs
USRFM = USRFWt/32.2/12 #Right Front Unsprung Mass 
USRWt = 300 #Left Rear Unsprung Weight, lbs
USRM = USRWt/32.2/12 #Rear Unsprung Mass 
LFStatic = 700 #Left Front Scale Weight, lbs
RFStatic = 700 #Right Front Scale Weight, lbs
LRStatic = 868 #Left Rear Scale Weight, lbs
RRStatic = 528 #Right Rear Scale Weight, lbs
DragCoef = .42 #Vehicle Drag Coef
FrontArea = 21 #Vehicle Frontal Area, ft^2
a = (Nose/100)*WB #Long dist from front axle to CG
b = WB-a #Long dist from rear axle to CG

Suspension Rates

In [3]:
# Tire Spring Rate
KtLF = 1800 #Left Front Tire Spring Rate, lb/in
KtRF = 2500 #Right Front Tire Spring Rate, lb/in
KtLR = 1800 #Left Rear Tire Spring Rate, lb/in
KtRR = 2500 #Right Rear Tire Spring Rate, lb/in

# Wheel Rate
KwLF = 140 #Left Front Wheel Rate, lb/in
KwRF = 120 #Right Front Wheel Rate, lb/in
KwLR = 125 #Left Rear Wheel Rate, lb/in
KwRR = 110 #Right Rear Wheel Rate, lb/in

# Shock Rate
KshLF = 400 #Left Front Shock Rate, lb/in
KshRF = 450 #Right Front Shock Rate, lb/in
KshLR = 404 #Left Rear Shock Rate, lb/in
KshRR = 250 #Right Rear Shock Rate, lb/in

# Roll Bar Rate
Kfarb = 250 #Front Rollbar Rate, lb/in
Krarb = 0 #Rear Rollbar Rate, lb/in

Circle Track Analyzer Outputs
Dive, in
Roll, deg
VehLFWt, lbs
VehRFWt, lbs
VehLRWt, lbs
VehRRWt, lbs
RPM
Torque (missing?)
LongAccel, g's
LatAccel, g's
Downforce, lbs
Speed, mph

Initial test with 5 timesteps to sort code. Should be able to read this file in.

In [27]:
Dive = [0.55, -0.21, -0.33, -0.42, -0.49] 
Roll = [0.42, 1.44, 1.4, 1.31, 1.23]
VehLFWt =[355, 369, 395, 418, 455]
VehRFWt = [986, 920, 851, 798, 764]
VehLRWt = [856, 925, 991, 1038, 1053]
VehRRWt = [906, 833, 751, 686, 634]
RPM = [4316, 4509, 4879, 5251, 5616]
LongAccel = [0, 0.52, 0.53, 0.52, 0.51]
LatAccel = [0.89, 0.88, 0.71, 0.52, 0.37]
Downforce = [317, 310, 246, 172, 115]
Speed = [67, 70, 76, 82, 88]

Perform preliminary calculations that will be used later based on Circle Track Analyzer Output
VertAccel, g's
Drag, lbs
LongForce, lbs
LatForce, lbs

In [28]:
# Vertical Acceleration

VertAccel = [Downforce[i]/VehWt for i in range(len(Downforce))]
#print(VertAccel)

# Convert Speed to ft/s
Speed_fps = [Speed[i]/(5280/3600) for i in range(len(Speed))]
#print(Speed_fps)

# Drag
Drag = [DragCoef*FrontArea*0.00258* Speed_fps[i]**2 for i in range(len(Speed_fps))]
#print(Drag)

# Longitudinal Force
Fx = [(LongAccel[i]*VehWt)+Drag[i] for i in range(len(LongAccel))]
#print(Fx)

# Lateral Force
Fy = [(LatAccel[i]*VehWt) for i in range(len(LatAccel))]
#print(Fy)

Calculate left front and right front instant center, anti dive and front view swing angle using previous regression coefficients. Moving forward, this should be regressed on the fly using dive and roll, or interpolated using dive.

Calculate left rear and right rear anti squat percentage. Moving forward this should be regressed on the fly using the dive and roll, or interpolated using dive.

In [23]:
# Left front instant center
LFIC = [6.3733 + -1.54 * Dive[i] + 0.3317 * Roll[i] + 0.1 * Dive[i]**2 + 0.015 * Roll[i]**2 + -0.08 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(LFIC)

# Left front anti dive
LFAD = [3.7976 + -0.0116 * Dive[i] + 0.1149 * Roll[i] + -0.042 * Dive[i]**2 + -0.0135 * Roll[i]**2 + 0.0613 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(LFAD)

# Left front front view swing angle
LFFVSA = [31.0644 + -1.521 * Dive[i] + 1.1083 * Roll[i] + 0.0802 * Dive[i]**2 + 0.0222 * Roll[i]**2 + -0.1025 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(LFFVSA)

# Right front instant center
RFIC = [6.7767 + -1.625 * Dive[i] + -0.3067 * Roll[i] + 0.115 * Dive[i]**2 + 0.01 * Roll[i]**2 + 0.08 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(RFIC)

# Right front anti dive
RFAD = [-2.4693 + -0.6151 * Dive[i] + -0.2664 * Roll[i] + -0.0652 * Dive[i]**2 + -0.0032 * Roll[i]**2 + -0.0297 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(RFAD)

# Right front front view swing angle
RFFVSA = [34.5054 + -2.2535 * Dive[i] + -1.4583 * Roll[i] + 0.1298 * Dive[i]**2 + 0.0363 * Roll[i]**2 + 0.158 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(RFFVSA)

# Left rear anti squat percentage
LRAS = [28.2053 + -19.5437 * Dive[i] + 12.8093 * Roll[i] + -1.2993 * Dive[i]**2 + 0.0467 * Roll[i]**2 + 0.9687 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(LRAS)

# Right rear anti squat percentage
RRAS = [66.4164 + -16.8635 * Dive[i] + -12.7975 * Roll[i] + -1.2193 * Dive[i]**2 + 0.0487 * Roll[i]**2 + -0.9245 * Dive[i] * Roll[i]
       for i in range(len(Dive))]
#print(RRAS)

[3.8385519, 3.91710908, 3.9029336, 3.8886875900000004, 3.87715714]
[-2.94664118, -2.72427456, -2.63892788, -2.5605938600000004, -2.49816861]
[22.675140329999998, 50.50147211, 52.09023983000001, 52.51180361, 52.71200701000001]
[51.19271792999999, 51.856116990000004, 54.45464422999999, 57.11149444999999, 59.276710449999996]


Longitudinal Braking, Driving, and Jacking Forces

In [38]:
# Front Longitudinal Ratio

LFLongRatio = (LFTW + CGy)/(LFTW+RFTW)
RFLongRatio = (RFTW - CGy)/(LFTW+RFTW)
#print(LFRatio, RFRatio)

# Rear Longitudinal Ratio

LRLongRatio = (LRTW + CGy)/(LRTW+RRTW)
RRLongRatio = (RRTW - CGy)/(LRTW+RRTW)
#print(LRRatio, RRRatio)

# Seperate longitudinal forces based on braking or acceleration

#Driving
#Drive = [Fx[i] for i in range(len(LongAccel)) if LongAccel[i] > 0 ]
#print(Drive)

#Braking
#Brake = [Fx[i] for i in range(len(LongAccel)) if LongAccel[i] > 0 ]
#print(Drive)

LFFx = []
RFFx = []
LRFx = []
RRFx = []

for i in range(len(LongAccel)):
    if LongAccel[i] >= 0: #Traction
        LFFx.append(0)
        RFFx.append(0)
        LRFx.append(LRLongRatio * Fx[i])
        RRFx.append(RRLongRatio * Fx[i])
    else: #Braking
        LFFx.append(LFLongRatio * Fx[i])
        RFFx.append(RFLongRatio * Fx[i])
        LRFx.append(LRLongRatio * Fx[i])
        RRFx.append(RRLongRatio * Fx[i])
print(LFFx, RFFx, LRFx, RRFx) 

#LF Longitudinal Jacking Force
LFLongJack = [LFFx[i] *(LFAD[i]/100) for i in range(len(LFFx))]
#print(LFLongJack)

#RF Lonitudinal Jacking Force
RFLongJack = [RFFx[i] *(RFAD[i]/100) for i in range(len(RFFx))]
#print(RFLongJack)

#LR Anti Squat Force
LRLongJack = [LRFx[i] *(LRAS[i]/100) for i in range(len(LFFx))]
#print(LRLongJack)

#RR Anti Squat Force
RRLongJack = [RRFx[i] *(RRAS[i]/100) for i in range(len(RFFx))]
#print(RRLongJack)

[0, 0, 0, 0, 0] [0, 0, 0, 0, 0] [27.51069468217226, 873.5349181912302, 895.124714650867, 884.7132445668778, 874.7430242082432] [19.97634021452198, 634.2998937922412, 649.976892291282, 642.4168117141141, 635.1771357917571]


Lateral Force and Lateral Jacking Force

In [34]:
#Front Lateral Force 
Fyf = [(a/100)*Fy[i] for i in range(len(Fy))]
#print(Fyf)

#Rear Lateral Force 
Fyr = [Fy[i]-Fyf[i] for i in range(len(Fy))]
#print(Fyr)

#Front Lateral Ratio
LFLatRatio = [VehLFWt[i]/VehWt for i in range(len(VehLFWt))]
RFLatRatio = [VehRFWt[i]/VehWt for i in range(len(VehRFWt))]

#Rear Lateral Ratio
LRLatRatio = [VehLRWt[i]/VehWt for i in range(len(VehLRWt))]
RRLatRatio = [VehRRWt[i]/VehWt for i in range(len(VehRRWt))]
#print(LFLatRatio, RFLatRatio, LRLatRatio, RRLatRatio)

#LF Lateral Force
LFFy = [LFLatRatio[i]*Fyf[i] for i in range(len(LFLatRatio))]

#RF Lateral Force
RFFy = [RFLatRatio[i]*Fyf[i] for i in range(len(RFLatRatio))]
#print(LFFy, RFFy)

#LF Lateral Jacking Force
LFLatJack = [(LFIC[i]/LFFVSA[i])*LFFy[i] for i in range(len(LFIC))]

#RF Lat Force
RFLatJack = [(RFIC[i]/RFFVSA[i])*RFFy[i] for i in range(len(RFIC))]
#print(LFLatJack, RFLatJack)

[31.56857660693381, 38.36872579250705, 33.8429851145696, 26.612785617003446, 20.8399241006347] [84.09178044641294, 88.70586974559342, 67.56070240068674, 47.10660557712366, 32.46596021288065]


Rear lateral forces and jacking forces manifest themselves in the sum of the moments around the LR Instant Center and RR Instant Center. The sum of the moments and the rear wheel radius are used to calculate the lateral affect on the LRFz and RRFz forces.

In [42]:
import math as m

#LR Instant Center
P2A = RWR-LRP
LRIC = RWR-P2A- (P2LR*m.tan(PAng))

#RR Instant Center
RRIC = ((LRTW+RRTW)*m.tan(PAng))+LRIC
#print(LRIC, RRIC)

#Sum moments around RR wheel radius to solve for LRFz
LRFz = [((LRLongJack[i]*(LRTW+LA2CL)) - (Fyr[i]*RRIC) + (RRLongJack[i]*(RRTW-LA2CL)))/(LRTW+RRTW)
       for i in range(len(Fyr))]

#Sum moments around LR wheel radius to solve for RRFz
RRFz = [((LRLongJack[i]*(LRTW-LA2CL)) + (Fyr[i]*LRIC) + (RRLongJack[i]*(RRTW+LA2CL)))/(LRTW+RRTW)
       for i in range(len(Fyr))]
#print(LRFz, RRFz)

[-259.45101041908174, 163.59464696375807, 239.55278787884288, 296.5600740636756, 339.6156517922287] [342.0045748377005, 671.8951095358655, 633.4567385411128, 573.5884593915134, 525.5197341134528]


Solve for aerodynamic forces

In [43]:
# Aerodynamic forces

LFAero = [(Downforce[i]*LFLongRatio)/2 for i in range(len(Downforce))]
RFAero = [(Downforce[i]*RFLongRatio)/2 for i in range(len(Downforce))]
LRAero = [(Downforce[i]*LRLongRatio)/2 for i in range(len(Downforce))]
RRAero = [(Downforce[i]*RRLongRatio)/2 for i in range(len(Downforce))]
print(LFAero, RFAero, LRAero, RRAero)

[91.82390765416798, 89.7962503873567, 71.257669662225, 49.82243569879146, 33.31151224047103] [66.67609234583205, 65.20374961264332, 51.74233033777503, 36.177564301208555, 24.188487759528975] [91.82390765416798, 89.7962503873567, 71.257669662225, 49.82243569879146, 33.31151224047103] [66.67609234583205, 65.20374961264332, 51.74233033777503, 36.177564301208555, 24.188487759528975]


Create Spring Matrix for [F] = [k] * [x] size of spring matrix is K(6,6)